In [1]:
# hide pakcage non critical warnings

import warnings

warnings.filterwarnings('ignore')

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))


%load_ext autoreload
%autoreload 2

%matplotlib inline

In [4]:
from easydict import EasyDict as edict



In [2]:
import matplotlib.pyplot as plt

import scipy.io as sio

import torch
import torch.nn as nn

import torch.nn.functional as F
import torch.nn.init as init

from custom_data import DCCPT_data

from config import cfg, get_data_dir, get_output_dir, AverageMeter, remove_files_in_dir
from convSDAE import convSDAE
from tensorboard_logger import Logger

import os
import random
import numpy as np

import data_params as dp

In [16]:
args = edict()
args.db = 'cmnist'
args.dim = 10

In [5]:
params = {'nlayers':4, 'dropout':0.2, 'reluslope':0.0,
          'nepoch':200, 'lrate':[10], 'wdecay':[0.0], 'step':20}

In [8]:
net_convAE = dp.load_predefined_net(args, params)

In [18]:
net_extract = dp.load_predefined_extract_net(args)

In [19]:
net_extract

extractconvSDAE(
  (enc_0): Conv2d(1, 50, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (benc_0): BatchNorm2d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dec_0): ConvTranspose2d(50, 1, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (bdec_0): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (enc_1): Conv2d(50, 50, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
  (benc_1): BatchNorm2d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dec_1): ConvTranspose2d(50, 50, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), output_padding=(1, 1))
  (bdec_1): BatchNorm2d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (enc_2): Conv2d(50, 50, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
  (benc_2): BatchNorm2d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dec_2): ConvTranspose2d(50, 50, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
  (bdec_2): Ba

In [10]:
from torchsummary import summary


In [12]:
summary?

In [21]:
summary(net_extract, ((1, 28, 28)))
print(net_extract)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 50, 14, 14]             850
       BatchNorm2d-2           [-1, 50, 14, 14]             100
            Conv2d-3             [-1, 50, 7, 7]          62,550
       BatchNorm2d-4             [-1, 50, 7, 7]             100
            Conv2d-5             [-1, 50, 4, 4]          62,550
       BatchNorm2d-6             [-1, 50, 4, 4]             100
            Linear-7                   [-1, 10]           8,010
   ConvTranspose2d-8             [-1, 50, 4, 4]           8,050
       BatchNorm2d-9             [-1, 50, 4, 4]             100
  ConvTranspose2d-10             [-1, 50, 7, 7]          62,550
      BatchNorm2d-11             [-1, 50, 7, 7]             100
  ConvTranspose2d-12           [-1, 50, 14, 14]          62,550
      BatchNorm2d-13           [-1, 50, 14, 14]             100
  ConvTranspose2d-14            [-1, 1,